This Notebook will collect KPI's of users choice on selected stocks from Sharpe Calculations

In [1]:
import time
import os 

start_time = time.time() # measuring run time

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.helpers import divide_chunks

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------




In [2]:
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Optional
import pandas as pd
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm
from functools import lru_cache
import yfinance as yf

Set up Logging, and Batch Processes to Aviod Kickbakcs

In [3]:
# Setup logging
logging.basicConfig(level=logging.INFO)

# %% [Function Definitions]
@lru_cache(maxsize=1000)
def fetch_ticker_info_cached(ticker_symbol: str) -> Dict[str, Optional[float]]:
    """
    Cached retrieval of ticker info from Yahoo Finance to minimize repeated API calls.
    """
    try:
        return yf.Ticker(ticker_symbol).info
    except Exception as e:
        logging.error(f"Error fetching data for {ticker_symbol}: {e}")
        return {}

@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=4, max=10)
)

def fetch_with_retry(ticker: str) -> Dict[str, Optional[float]]:
    """
    Retry fetching data for a ticker symbol using tenacity.
    """
    return fetch_ticker_info_cached(ticker)

def get_stock_kpis(ticker: str, kpis: List[str]) -> Dict[str, Optional[float]]:
    """
    Fetch KPIs for a specific stock ticker.
    """
    try:
        info = fetch_with_retry(ticker)
        kpi_data = {kpi: info.get(kpi) for kpi in kpis}
        kpi_data['Tickers'] = ticker  # Add ticker column
        return kpi_data
    except Exception as e:
        logging.error(f"Error processing {ticker}: {e}")
        return {'Tickers': ticker, **{kpi: None for kpi in kpis}}


def batch_process_stocks(tickers: List[str], kpis: List[str], max_workers: int = 10) -> pd.DataFrame:
    """
    Process stock tickers in batches using ThreadPoolExecutor for parallel API calls.
    """
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(get_stock_kpis, ticker, kpis): ticker
            for ticker in tickers
        }

        for future in tqdm(as_completed(future_to_ticker), total=len(tickers)):
            results.append(future.result())

    # Create DataFrame from results
    df = pd.DataFrame(results)
    return df


In [4]:
# Find Benchmark Sharpe
# # Load the Sharpe ratio DataFrame

sharpe_ratio_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/sharpe_ratios.csv')
spy_sharpe = sharpe_ratio_df.loc[sharpe_ratio_df['Tickers'] == 'SPY', 'Sharpe_ratios'].values[0]

# Filter only stocks above benchmark
sharpe_ratio_df = sharpe_ratio_df[sharpe_ratio_df['Sharpe_ratios'] >= spy_sharpe]

print(f'Spy Sharpe: {spy_sharpe.round(2)}')

Spy Sharpe: 1.92


In [5]:
num_in_chunks = 50
tickers = sharpe_ratio_df['Tickers'].tolist()

print('\n----Intiating Chunk Process----')
ticker_chunks = list(divide_chunks(tickers, num_in_chunks))
print(f'{len(tickers)} tikcers divided into {len(ticker_chunks)} chunks')
        
essential_kpis = ['marketCap', 'trailingPE', 'profitMargins', 'trailingEps']

df_list = []

# Loop through each chunk and process the stocks
print('\n----Fethcing KPIs----')
for chunk in ticker_chunks:
    kpi_df_chunk = batch_process_stocks(chunk, essential_kpis)
    df_list.append(kpi_df_chunk)

# Concatenate all DataFrames vertically
kpi_df = pd.concat(df_list, axis=0, ignore_index=True)
print(f'KPI Df Shape: {kpi_df.shape}')


----Intiating Chunk Process----
71 tikcers divided into 2 chunks

----Fethcing KPIs----


100%|██████████| 21/21 [00:00<00:00, 71.83it/s]

KPI Df Shape: (71, 5)


In [6]:

print('\n----Combining Sharpe & KPI Df----')
# Combine results with Sharpe ratio data
valuation_df = pd.merge(kpi_df, sharpe_ratio_df, on='Tickers').dropna()
print('\n----Filtering Df----')
# Filter and save
filtered_valuation_df = valuation_df[
    (valuation_df['profitMargins'] > 0.2) &
    (valuation_df['Sharpe_ratios'] > 0.75) &
    (valuation_df['marketCap'] > 10_000_000_000)
].sort_values('profitMargins', ascending=False)

filtered_valuation_df.to_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv', index=False)
logging.info(f"Final filtered DataFrame saved. Rows: {filtered_valuation_df.shape[0]}")

# Print the sorted DataFrame (Optional)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'\nTotal Run Time: {elapsed_time} seconds')
filtered_valuation_df

INFO:root:Final filtered DataFrame saved. Rows: 19



----Combining Sharpe & KPI Df----

----Filtering Df----

Total Run Time: 1.5529170036315918 seconds


,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios
62,3.173270e+10,70.93939,0.65343,19.47,TPL,2.160594
5,3.271130e+12,52.586617,0.55041,2.54,NVDA,2.578982
22,9.102460e+10,8.232416,0.50511,6.54,MO,2.163501
66,2.970253e+10,16.979937,0.41335,12.46,GDDY,3.328520
14,1.514610e+11,56.71227,0.40294,2.12,ANET,2.047055
46,5.448069e+10,12.634435,0.39410,4.24,MPLX,2.742126
70,2.547090e+10,7.893975,0.35139,8.30,SYF,2.822645
54,4.506030e+10,8.68504,0.34031,1.27,NWG,2.779077
45,4.535296e+10,85.474,0.29858,21.73,FICO,2.238668
47,4.554514e+10,23.996492,0.28840,2.85,TCOM,1.981917
